In [64]:
import pandas as pd
import re

In [65]:
ml = pd.read_csv("/home/mario/Proyectos_Mario/proyecto_alonso/datos_extraidos/casas_mercadoLibre.csv")

In [66]:
# Se tienen algunos valores nulos
ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          23 non-null     object 
 1   seller         23 non-null     object 
 2   property_type  12 non-null     object 
 3   address        22 non-null     object 
 4   price          23 non-null     int64  
 5   bedrooms       23 non-null     int64  
 6   bathrooms      23 non-null     int64  
 7   built_Area     23 non-null     float64
 8   land_Area      23 non-null     int64  
 9   parking        23 non-null     int64  
 10  description    23 non-null     object 
dtypes: float64(1), int64(5), object(5)
memory usage: 2.1+ KB


In [67]:
# Se eliminan los valores nulos de la columna address
no_address = ml[ml.address.isnull()].index
ml = ml.drop(no_address)
ml = ml.reset_index(drop="first")

In [68]:
# Adecuo los valores de la columna adddress para hacer una búsqueda efectiva de los zip codes
ml["address"] = ml["address"].apply(lambda x: x.lower()\
                                                        .replace("á", "a")\
                                                        .replace("é", "e")\
                                                        .replace("í", "i")\
                                                        .replace("ó", "o")\
                                                        .replace("ú", "u"))
ml["address"] = ml["address"].apply(lambda x: x.replace(
                                                "agencia municipal de san felipe del agua", 
                                                "san felipe del agua"))
ml["address"] = ml["address"].apply(lambda x: x.replace("sierra de juarez",
                                                        "san felipe del agua"))

In [69]:
casas_remate = list(ml[ml.description.str.contains("remate|bancaria|recuperación|hipotecaria", case=False)].index)
ml = ml.drop(index=casas_remate)
ml = ml.reset_index(drop="first")

In [70]:
# Solo se tienen valores nulos en la columna property_type
ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          16 non-null     object 
 1   seller         16 non-null     object 
 2   property_type  6 non-null      object 
 3   address        16 non-null     object 
 4   price          16 non-null     int64  
 5   bedrooms       16 non-null     int64  
 6   bathrooms      16 non-null     int64  
 7   built_Area     16 non-null     float64
 8   land_Area      16 non-null     int64  
 9   parking        16 non-null     int64  
 10  description    16 non-null     object 
dtypes: float64(1), int64(5), object(5)
memory usage: 1.5+ KB


In [71]:
# Elimino una fila cuyo valor no sirve para el análisis ya que es una 
# propiedad pensada para uso comercial y no una casa
ml = ml.drop(index=2)
ml = ml.reset_index(drop="first")

In [72]:
# Se cambian los valores de property type ya que el resto de los 
# inmuebles son casas
ml["property_type"] = "Casa"

### Creando la columna parking

In [73]:
# Se tienen 9 filas que marcan como 0 el valor de parking
len(ml.query("parking==0"))

9

In [74]:
# Con esta función localizo el texto donde especifica los lugares de estacionamiento en la descripción
def econtrar_textos(texto):
    regex = r"(?i)(.{0,4}estacionamientos?.{0,20}|.{0,4}cocheras?.{0,20}|.{0,4}garages?.{0,20}|.{0,4}parkings?.{0,20})"
    expresion = re.findall(regex, texto)
    expresion = "".join(expresion)
    if len(expresion) > 0:
        valor_de_celda = expresion
    else:
        valor_de_celda = "cero"
    return valor_de_celda

In [75]:
# Función para encontrar en la descripción el número de lugares de estacionamiento
def asignar_valor_estacionamiento(texto):
    # Buscar dígitos numéricos en el texto
    if texto != "cero":    
        numeros_encontrados = re.findall(r'\d+', texto)

        if numeros_encontrados:
            # Tomar el primer número encontrado y convertirlo a entero
            return int(numeros_encontrados[0])
        elif "para dos" in texto.lower():
            return 2
        elif "para tres" in texto.lower():
            return 3
        else:
            return 1

In [76]:
# Aplicar las funciones para buscar los valores de parking
ml["estacionamiento"] = ml["description"].apply(econtrar_textos)
ml["estacionamiento"] = ml["estacionamiento"].apply(asignar_valor_estacionamiento)

In [77]:
# Reemplazar los valores en las filas de la columna parking donde marca cero
ml.loc[7:12, "parking"] = ml.loc[7:12, "estacionamiento"]

# Eliminar la columna estacionamiento
ml = ml.drop(columns="estacionamiento")

In [78]:
# Eliminar las filas donde parking es cero
no_parking = list(ml[ml["parking"]==0].index)
ml = ml.drop(index=no_parking)
ml = ml.reset_index(drop="first")

### Creando columna zip_code

In [79]:
# Leo la tabla de códigos postales
tabla_CP = pd.read_html("https://codigo-postal.co/mexico/oaxaca/oaxaca-de-juarez/")
tabla_CP = tabla_CP[0]

# Seleccionar las columas que me sirven 
tabla_CP = tabla_CP.iloc[:, :2]
tabla_CP = tabla_CP.drop_duplicates()

# Guardar en listas las colonias y sus C.P.
cp = list(tabla_CP.iloc[:, 0])
direccion = list(tabla_CP.iloc[:, 1])

# Crear un diccionario y Eliminar acentos de los nombres de las direcciones
cp_dict = {}
for i, j in zip(direccion, cp):
    cp_dict[i.lower()\
        .replace("á", "a")\
        .replace("é", "e")\
        .replace("í", "i")\
        .replace("ó", "o")\
        .replace("ú", "u")] = j

In [80]:
# Creo una función que asigna el zip code según la dirección
def buscador_zip_code(lista, diccionario):
    lista_coincidencias = []
    for i in lista:
        if (i.strip() == 'san felipe') | (i.strip() == "tulipanes"):
            pass
        else:
            regex = re.compile(fr"(.{{0,30}}{i.strip()}.{{0,30}})", re.IGNORECASE)
            coincidencia = list(filter(lambda x: regex.findall(x), diccionario.keys()))
            if len(coincidencia) > 0:
                lista_coincidencias.append(coincidencia[0])
    try:
        return diccionario["".join(lista_coincidencias[0])]
    except:
        None

In [81]:
# Separo los datos contenidos en dirección para poder trabajar con ellos
ml["zip_code"] = ml["address"].apply(lambda x: x.split(","))
# Asigno el zip code usando mi función buscador_zip_code
ml["zip_code"] = ml["zip_code"].apply(buscador_zip_code, diccionario=cp_dict)

In [82]:
# Se eliminan valores nulos y valores duplicados
ml = ml.drop_duplicates()
ml = ml.dropna(ignore_index=True)

In [83]:
ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          8 non-null      object 
 1   seller         8 non-null      object 
 2   property_type  8 non-null      object 
 3   address        8 non-null      object 
 4   price          8 non-null      int64  
 5   bedrooms       8 non-null      int64  
 6   bathrooms      8 non-null      int64  
 7   built_Area     8 non-null      float64
 8   land_Area      8 non-null      int64  
 9   parking        8 non-null      float64
 10  description    8 non-null      object 
 11  zip_code       8 non-null      int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 896.0+ bytes


In [84]:
ml.to_csv("datos_mercadoLibre.csv", index=False)